적대적 신경망(GAN)

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

생성자(Generator)

In [2]:
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
#128 : 임의의 노드 수 (설정 값)/ 7*7 (이미지 최초크기) >> 14*14 >> 28*28 upsampling
generator.add(BatchNormalization())
#배치정규화 : input data를 평균0 분산1이 되게 재배치
generator.add(Reshape((7,7,128)))
generator.add(UpSampling2D()) # 14*14로 변환
generator.add(Conv2D(64, kernel_size=5, padding='same'))
# kernel_size = 5 (5,5)padding='same' : zero padding
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D()) # 28*28로 변환
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh')) #tanh -1 ~ 1 사이로 출력


판별자(discriminator)

In [3]:
#feature extraction(특징 추출)

discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding='same'))
#strides=2 >> pooling 과정 대체 효과
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))

discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))

discriminator.add(Flatten()) # 2차원 >> 1차원
discriminator.add(Dense(1, activation='sigmoid'))

In [6]:
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False
#판별자 자신이 학습이 되지 않게 학습 기능을 꺼줌 (***)

## GAN NETWORK (생성자와 판별자 모델 연결)

In [8]:
ginput = Input(shape = (100,))
#무작위로 100개 벡터값 집어 넣음

# D() : 판별자 G() : 생성자
dis_output = discriminator(generator(ginput))
#생성된 28*28 이미지 >> 판별자의 입력값을 들어감

gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam') # 참/ 거짓 분류
gan.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         865281    
                                                                 
 sequential_1 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1078146 (4.11 MB)
Trainable params: 852609 (3.25 MB)
Non-trainable params: 225537 (881.00 KB)
_________________________________________________________________
